In [1]:
import numpy as np
from IPython.display import clear_output

In [2]:
data_direct = np.load('../data/direct_citations.npy', allow_pickle=True)

In [3]:
BERT_PAPER_ID = 'df2b0e26d0599ce3e70df8a9da02e51594e0e992'

In [4]:
direct_cit_ids = [cit['citingPaper']['paperId'] for cit in data_direct]

In [5]:
len(direct_cit_ids)

9999

In [6]:
direct_cit_ids = list(set(direct_cit_ids))

In [7]:
data_indirect = np.load('../data/indirect_citations.npy', allow_pickle=True)

In [8]:
bert_cit = {BERT_PAPER_ID : direct_cit_ids}

In [9]:
data_indirect = np.delete(data_indirect, [1530, 5194])

In [10]:
cits = []
for sample in data_indirect:
    key = list(sample.keys())[0]
    d = {key: []}
    for cit in sample[key]:
        d[key].append(cit['citingPaper']['paperId'])
    cits.append(d)

In [11]:
len(bert_cit[BERT_PAPER_ID])

9994

In [12]:
for cit in cits:
    key = list(cit.keys())[0]
    bert_second_level_cits = cit[key]
    for s_l_cit in bert_second_level_cits:
        if s_l_cit in bert_cit[BERT_PAPER_ID]:
            bert_cit[BERT_PAPER_ID].pop(bert_cit[BERT_PAPER_ID].index(s_l_cit))

In [13]:
len(bert_cit[BERT_PAPER_ID])

5220

In [14]:
cits.append(bert_cit)

In [15]:
unique_ids = []
for cit in cits:
    key = list(cit.keys())[0]
    unique_ids.append(key)
    unique_ids.extend(cit[key])

In [16]:
unique_ids = list(set(unique_ids))

In [17]:
len(set(unique_ids))

27444

In [18]:
faulty = [
     '80cbe3f92d5fb20e5fad53b4659b9bcb5733ad4a',
     '3fd0f34117cf9395130e08c3f02ac2dadcca7206',
     '78b3ad1123cf693c74b1be0097f3f1a8ecb2ace3',
     '2e3196e4a71338816b5c405a38d137faf18209ef',
     '78590d84bbbd46176262c104ad728e492389a4dd',
     '752e19ada610149f5b1fe679f9ab3e00a3500c15',
     'c143a87231aa55d2c1718e476725861c7e368828',
     '5e1657676c7c486f314cbdf183e26b4c1405ca1b',
     '1225570385519c03b9f9ee9b1e5a232de6228db3',
     'ce693eac698fe6c6701be2e2b5b34f466be3b8ad',
     '56d57505e2a491d49a69a9fa82bca76e14d4398c',
     'f2d3a5f58192f4fa4fd85d16989329393b2591a2',
     '6cfda665faa5d761b83b20ccd988a90b6e6b1c88',
     '262c4b90243fd31fa721da0f4f875c39e08e47f3',
     '25248231feeb950a88915804733a0b736c7af93d'
]

In [19]:
absless_ids = list(np.load('../data/abstractless_keys.npy', allow_pickle=True))

In [20]:
for f in list(set(faulty + absless_ids)):
    if f in unique_ids:
        unique_ids.pop(unique_ids.index(f))

In [21]:
len(set(unique_ids))

24828

In [22]:
np.save('../data/unique_ids_list.npy', np.array(list(set(unique_ids))))

In [25]:
indexes = []
for n, ent in enumerate(cits):
    key = list(ent.keys())[0]
    if key not in unique_ids:
        indexes.append(n)

In [27]:
cits = np.delete(np.array(cits), indexes)

In [29]:
cits = list(cits)

In [32]:
for ent in cits:
    key = list(ent.keys())[0]
    tmp = []
    refs = ent[key]
    for ref in refs:
        if ref in unique_ids:
            tmp.append(ref)
    ent[key] = tmp

In [36]:
np.save('../data/citation_relations.npy', np.array(cits))